In [1]:
import tensorflow as tf
import os

In [2]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string("cifar_dir", "./data/cifar10/cifar-10-batches-bin", "文件的目录")
tf.app.flags.DEFINE_string("cifar_tfrecords", "./data/cifar10/cifar.tfrecords","存进tfrecords的文件")
tf.app.flags.DEFINE_string("f", "","")


In [4]:
class CifarRead(object):
    """
    完成读取二进制文件，写进tfrecords，读取tfrecords
    """
    
    def __init__(self, filelist):
        self.filelist = filelist
        
        # 定义读取的图片的一些属性
        self.height = 32
        self.width = 32
        self.channel = 3
        
        # 二进制文件每张图片的字节
        self.label_bytes = 1
        self.image_bytes = self.height * self.width * self.channel
        self.bytes = self.label_bytes + self.image_bytes
        
    def read_and_decode(self):
        # 1. 构造文件队列
        file_queue = tf.train.string_input_producer(self.filelist)
        
        # 2. 构造二进制文件读取器，读取内容:每个样本的字节数
        reader = tf.FixedLengthRecordReader(self.bytes)
        key, value = reader.read(file_queue)
        
        # 3.解码内容
        label_image = tf.decode_raw(value, tf.uint8)
        
        print(label_image)
        
        # 4.分割出图片和标签数据，切出特征值和目标值
        label = tf.slice(label_image, [0], [self.label_bytes])
        image = tf.slice(label_image, [self.label_bytes], [self.image_bytes])
#         a = tf.reshape(label_image[0:1], [1])
#         b = tf.reshape(label[1:], [self.image_bytes])
        print(label, image)
#         print(a, b)
        print("*"*20)
        # 5. 对图片的特征数据进行形状改变 [3072] -> [32, 32, 3]
        image_reshape = tf.reshape(image, [self.height, self.width, self.channel])
        
        print(label, image_reshape)
        
        # 6. 批处理
        image_batch, label_batch = tf.train.batch([image_reshape, label],
                                                 batch_size =10,
                                                 num_threads = 1,
                                                 capacity=10)
        print(image_batch, label_batch)
        return image_batch, label_batch
    

    def write_to_tfrecords(self, image_batch, label_batch):
        """
        将图片的特征值和目标值存进tfrecords
        image_batch: 10张图片的特征值
        label_batch: 10张图片的标签值
        """
        
        # 1. 建立tfrecords存储器
        writer = tf.python_io.TFRecordWriter(FLAGS.cifar_tfrecords)
    
        # 2. 循环将所有样本写入文件，每张图片样本都要构造example协议
        for i in range(10):
            # 取出第i个图片数据的特征值和目标值
            # image_batch shape [10, 32, 32, 3]
            # label_batch shape [10,1]
            image = image_batch[i].eval().tostring()
            label = label_batch[i].eval()[0]
        
            # 构造一个样本的example
            example = tf.train.Example(features=tf.train.Features(feature={
                "image":tf.train.Feature(bytes_list=
                                        tf.train.BytesList(value=[image])),
                "label":tf.train.Feature(int64_list=
                                        tf.train.Int64List(value=[label])),
            }))
            
            # 写入单独的样本
            writer.write(example.SerializeToString())
            
        writer.close()
        return None

    def read_from_tfrecords(self, file_list):
        
        #1. 构造文件队列
        file_queue = tf.train.string_input_producer(file_list)
        
        # 2. 构造文件阅读器，读取内容，value=一个样本的序列化example
        reader = tf.TFRecordReader()
        
        key, value = reader.read(file_queue)
        
        # 3. 解析 example
        features = tf.parse_single_example(value, features={
            "image": tf.FixedLenFeature([], tf.string),
            "label": tf.FixedLenFeature([], tf.int64),
        })
        
        # 4. 解码内容，如果读取的内容格式是string，需要解码
        image = tf.decode_raw(features["image"], tf.uint8)
        
        # 固定图片的形状，方便批处理
        image_reshape = tf.reshape(image,[self.height,self.width, self.channel])
        label = tf.cast(features["label"], tf.int32)
        
        print(image_reshape, label)
        
        #批处理
        image_batch, label_batch = tf.train.batch([image_reshape,label],
                                                 batch_size=10,
                                                 num_threads=1,
                                                 capacity=10)
        
        return image_batch, label_batch
        

In [24]:
# file_name = os.listdir(FLAGS.cifar_dir)
# print(file_name)

# [i for i in file_name if i[-4:] == ".bin" and i[:10]=="data_batch"]

In [6]:
# 找到文件，放入列表， 路径+名字 
file_name = os.listdir(FLAGS.cifar_dir)
file_list = [os.path.join(FLAGS.cifar_dir, i) for i in file_name
            if i[-4:]==".bin" and i[:4]=="data"]

cf = CifarRead(file_list)

# 1. read from binary
image_batch, label_batch = cf.read_and_decode()

# 2. read from tfrecords
# image_batch, label_batch = cf.read_from_tfrecords([FLAGS.cifar_tfrecords])



#开启会话运行结果
with tf.Session() as sess:
    # 定义一个线程协调器
    coord = tf.train.Coordinator()
    
    #开启读文件的线程
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
    # 存进tfrecords 文件
    cf.write_to_tfrecords(image_batch, label_batch)
    
#     print(sess.run([image_batch ]))
    print(sess.run([ label_batch]))
    
    coord.request_stop()
    coord.join(threads)

Tensor("DecodeRaw_1:0", shape=(?,), dtype=uint8)
Tensor("Slice_2:0", shape=(1,), dtype=uint8) Tensor("Slice_3:0", shape=(3072,), dtype=uint8)
********************
Tensor("Slice_2:0", shape=(1,), dtype=uint8) Tensor("Reshape_1:0", shape=(32, 32, 3), dtype=uint8)
Tensor("batch_1:0", shape=(10, 32, 32, 3), dtype=uint8) Tensor("batch_1:1", shape=(10, 1), dtype=uint8)
[array([[5],
       [5],
       [7],
       [3],
       [3],
       [0],
       [4],
       [5],
       [3],
       [8]], dtype=uint8)]


In [53]:
p = tf.placeholder(tf.float32, [None])
p.set_shape([1])
p

<tf.Tensor 'Placeholder_7:0' shape=(1,) dtype=float32>

In [41]:
a = tf.constant(list(range(10)))
b = a[3:]
c = tf.slice(a, [0], [5])
print(a)
print(b)
print(c)
with tf.Session() as sess:
    print(a.eval().tostring())
    print(a)
    

Tensor("Const_5:0", shape=(10,), dtype=int32)
Tensor("strided_slice_18:0", shape=(7,), dtype=int32)
Tensor("Slice_12:0", shape=(5,), dtype=int32)
b'\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00'
Tensor("Const_5:0", shape=(10,), dtype=int32)


In [ ]:
import pickle
a = list(range(10))
print(a)
pickle.dumps(a)

In [85]:
# tostring test
import numpy as np
a = np.arange(12,dtype='int32').reshape((3,4))
                         
print(a.tostring())

b = tf.constant(a)
with tf.Session() as sess:
    print(b.eval().tostring())

b'\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00\n\x00\x00\x00\x0b\x00\x00\x00'
b'\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00\n\x00\x00\x00\x0b\x00\x00\x00'


In [ ]:
a = np.arange(10,dtype='int32').reshape((10,1))
a